## Get Qubit Hamiltonian

In [2]:
from quchem.Hamiltonian_Generator_Functions import *

### Parameters
Molecule = 'H2'#LiH'
geometry = None
num_shots = 10000
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!

Hamilt.Get_Molecular_Hamiltonian()
SQ_CC_ops, THETA_params = Hamilt.Get_ia_and_ijab_terms(Coupled_cluser_param=True)
#print('UCC operations: ', SQ_CC_ops)

HF_transformations = Hamiltonian_Transforms(Hamilt.MolecularHamiltonian, SQ_CC_ops, Hamilt.molecule.n_qubits)

QubitHam = HF_transformations.Get_Qubit_Hamiltonian_JW(threshold=None) # threshold=1e-12
#print('Qubit Hamiltonian: ', QubitHam)
QubitHam_PauliStr = HF_transformations.Convert_QubitMolecularHamiltonian_To_Pauliword_Str_list(QubitHam, Hamilt.molecule.n_qubits)

### Graph Colouring
from quchem.Graph import *
List_PauliWords, HamiltonainCofactors = zip(*QubitHam_PauliStr)
attribute_dictionary = {'Cofactors': HamiltonainCofactors}
List_of_nodes, node_attributes_dict = Get_list_of_nodes_and_attributes(List_PauliWords,
                                                                       attribute_dictionary=attribute_dictionary)
G = Hamiltonian_Graph(List_PauliWords, attribute_dictionary=attribute_dictionary)
anti_commuting_sets = G.Get_Pauli_grouping('AC', Graph_colouring_strategy='largest_first', plot_graph=False)

anti_commuting_set_stripped = Get_PauliWord_constant_tuples(anti_commuting_sets, dict_str_label='Cofactors')
anti_commuting_set_stripped


Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 16362.69it/s]


{0: [('I0 I1 I2 I3', (-0.32760818995565577+0j))],
 1: [('Z0 Z1 I2 I3', (0.15660062486143395+0j))],
 2: [('Z0 I1 Z2 I3', (0.10622904488350779+0j))],
 3: [('Z0 I1 I2 Z3', (0.15542669076236065+0j))],
 4: [('I0 Z1 Z2 I3', (0.15542669076236065+0j))],
 5: [('I0 Z1 I2 Z3', (0.10622904488350779+0j))],
 6: [('I0 I1 Z2 Z3', (0.1632676867167479+0j))],
 7: [('Z0 I1 I2 I3', (0.1371657293179602+0j)),
  ('Y0 X1 X2 Y3', (0.04919764587885283+0j))],
 8: [('I0 Z1 I2 I3', (0.1371657293179602+0j)),
  ('Y0 Y1 X2 X3', (-0.04919764587885283+0j))],
 9: [('I0 I1 Z2 I3', (-0.13036292044009176+0j)),
  ('X0 X1 Y2 Y3', (-0.04919764587885283+0j))],
 10: [('I0 I1 I2 Z3', (-0.13036292044009176+0j)),
  ('X0 Y1 Y2 X3', (0.04919764587885283+0j))]}

## Get Ansatz

In [ ]:
from quchem.Ansatz_Generator_Functions import *

UCCSD_JW = UCCSD_Trotter_JW(SQ_CC_ops, THETA_params)
Second_Quant_CC_JW_OP_list = UCCSD_JW.SingleTrotterStep()

PauliWord_list = Convert_QubitOperator_To_Pauliword_Str_list(Second_Quant_CC_JW_OP_list, Hamilt.molecule.n_qubits)
HF_UCCSD_ansatz = Ansatz_Circuit(PauliWord_list, Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)


ansatz_Q_cicuit = HF_UCCSD_ansatz.Get_Full_HF_UCCSD_QC(THETA_params)



### alt form

from quchem.quantum_circuit_functions import *
from quchem.Ansatz_Generator_Functions import HF_state_generator
HF_state_obj = HF_state_generator(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
HF_state = HF_state_obj.Get_JW_HF_state_in_occ_basis()

def NEW_Anastz(theta):

    initial_state = State_Prep(HF_state)
    HF_circuit = cirq.Circuit(
        cirq.decompose_once((initial_state(*cirq.LineQubit.range(initial_state.num_qubits())))))

    Pauli = ('Y0 X1 X2 X3',  -1j)
    circuit_obj = full_exponentiated_PauliWord_circuit(Pauli, theta)
    circuit_exp = cirq.Circuit(
        cirq.decompose_once((circuit_obj(*cirq.LineQubit.range(circuit_obj.num_qubits())))))

    full_Circuit = cirq.Circuit(
       [
           HF_circuit.all_operations(),
           *circuit_exp.all_operations(),
       ]
    )

    return full_Circuit


ansatz_Q_cicuit = NEW_Anastz(np.pi)


## Looking at LCU method

In [ ]:
from quchem.LCU_method import *

test_set = anti_commuting_set_stripped[7]
S_index=0

S = set
$$ S =\sum_{k=0}^{n-1} \beta_{j} P_{j}$$
where all terms anti-commute: $ \{P_{j}P_{i},P_{i}P_{j}\}=0$

choosing a PauliWord in set: $ P_{n}$ and REMOVE IT

$$ S_{n-1} =\sum_{k=1}^{n-1} \beta_{j} P_{j}$$

note sum index starting from 1 now!

Next need amplitudes to sum to one... therefore have:

$$ S'_{n-1} = \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{k}$$

where:
- $ \beta_{j} = \gamma_{l} \times \beta_{k}$ 
- $ \sum_{k=1}^{n-1} \beta_{k}^{2} = 1$

NOW we define $H_{n-1}$:

$$ H_{n-1} =  \frac{S'_{n-1}}{\gamma_{l}} =  \sum_{k=1}^{n-1} \beta_{k} P_{k}$$



$$\chi =i H_{n-1} P_{n} =i \big( \sum_{k=1}^{n-1} \beta_{k} P_{k} \big) P_{n} = i \sum_{k=1}^{n-1} \beta_{k} P_{kn}$$


$$R=e^{\big(-i \frac{\alpha}{2} \mathcal{X} \big)}$$

$$R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X})$$


$$R H_{n} R^{\dagger}= \sin \left(\phi_{n-1}-\alpha \right) H_{n-1}+\cos \left(\phi_{n-1}-\alpha \right) P_{n}$$

$$R H_{n} R^{\dagger}= \sin \left(\phi_{n-1}-\alpha \right) \frac{S'_{n-1}}{\gamma_{l}} +\cos \left(\phi_{n-1}-\alpha \right) P_{n}$$

THEN choosing: $\alpha = \phi_{n-1}$ gives: $R H_{n} R^{\dagger}=P_{n}$

To find alpha it is simply:
$$\alpha = cos^{-1}(\beta_{n})$$


NOTE it seems that **$\gamma_{l}$** term isn't important as whole term goes to ZERO

## shorter circuit


$$R H_{n} R^{\dagger}= \sin \left(\phi_{n-1}-\alpha \right) \frac{S'_{n-1}}{\gamma_{l}} +\cos \left(\phi_{n-1}-\alpha \right) P_{n}$$

choosing: $\alpha = \phi_{n-1}$ gives:

$$R H_{n} R^{\dagger}=  P_{n}$$

$$R H_{n} =  P_{n} R$$

note:
- $R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X})$
- $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$
    - where $ H_{n-1} =  \frac{S'_{n-1}}{\gamma_{l}} =  \sum_{k=1}^{n-1} \beta_{k} P_{k}$

### iteratively find RH_n

H_n is just a summation of pauli terms from 0 to S (S=size of anti_commuting set)

$$H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n} = \sum_{q=0}^{S-1} \beta_{q} P_{q}$$

R is also just a summation of Pauli terms:

$$R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X}) =  \sum_{r=0}^{S-1} \beta_{r} P_{r}$$


so: 

$$R H_{n} = \sum_{r=0}^{S-1} \beta_{r} P_{r} * \sum_{q=0}^{S-1} \beta_{q} P_{q}$$

$$R H_{n} = \sum_{r=0}^{S-1} \sum_{q=0}^{S-1} \beta_{r}  \beta_{q} P_{r} P_{q}$$

^^ what we do in code

$$R H_{n} = \sum_{rq=0}^{S-1}\beta_{rq} P_{rq}$$

rather than iteratively doing... see analytical approach:

### analytical approach to find RH_n

$$R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X})$$

$$R H_{n} =  P_{n} R$$

$$R H_{n} = \bigg( \cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X}\bigg) H_{n}$$

$$R H_{n} = \cos \big( \frac{\alpha}{2} \big) H_{n} -i \sin \big(\frac{\alpha}{2}\big) \mathcal{X}H_{n}$$

as:
- $\mathcal{X} H_{n}=i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$
- $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$
- AND **IMPORTANTLY**: $\alpha = \phi_{n-1}$ 

  $$R H_{n} = \cos \big( \frac{\alpha}{2} \big) \bigg(\sin (\alpha) H_{n-1}+\cos (\alpha) P_{n} \bigg) -i \sin \big(\frac{\alpha}{2}\big) \bigg(i (-\sin (\alpha) P_{n}+\cos (\alpha) H_{n-1}) \bigg)$$


$$R H_{n} = \cos \big( \frac{\alpha}{2} \big)\sin (\alpha) H_{n-1}+ \cos \big( \frac{\alpha}{2} \big)\cos (\alpha) P_{n} - \sin \big(\frac{\alpha}{2}\big)  \sin (\alpha) P_{n} + \sin \big(\frac{\alpha}{2}\big)  \cos (\alpha) H_{n-1})$$

$$R H_{n} = H_{n-1}\bigg(
 \cos \big( \frac{\alpha}{2} \big)\sin (\alpha)
 +
 sin \big(\frac{\alpha}{2}\big)  \cos (\alpha)
 \bigg)
+ P_{n}\bigg(
\cos \big( \frac{\alpha}{2} \big)\cos (\alpha)
-
\sin \big(\frac{\alpha}{2}\big)  \sin (\alpha)
\bigg)$$

Using the following identities:

- $ \cos \big( \frac{\alpha}{2} \big)\sin (\alpha)+sin \big(\frac{\alpha}{2}\big)  \cos (\alpha) = \sin \big( \frac{3\alpha}{2} \big)$
- $ \cos \big( \frac{\alpha}{2} \big)\cos (\alpha)-\sin \big(\frac{\alpha}{2}\big)  \sin (\alpha) = \cos \big( \frac{3\alpha}{2} \big)$

$$R H_{n} = \sin \big( \frac{3\alpha}{2} \big) H_{n-1} + \cos \big( \frac{3\alpha}{2} \big)P_{n}$$

NOTE

for $\langle R H_{n} R^{\dagger} \rangle=  \langle P_{n} \rangle$ we are actually measuring:

$$\langle R H_{n} \rangle =   \langle P_{n} R \rangle$$

## $P_{n} R$ part -> What WE MEASURE!

$$R H_{n} =  P_{n} R$$

- where $R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X})$

$$ P_{n} R =  P_{n} \bigg( \cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X} \bigg)$$

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} -i \sin \big(\frac{\alpha}{2}\big) P_{n}\mathcal{X} $$

///////////////////////////////////////////////////////////

we want to know what: $P_{n} \mathcal{X} =??$

start with definition of $\mathcal{X}=\frac{i}{2}\left[H_{n-1}, P_{n}\right]$

$$\mathcal{X}=\frac{i}{2}\left[H_{n-1}, P_{n}\right] =\frac{i}{2} \big( H_{n-1} P_{n} - P_{n}  H_{n-1}\big)$$

$$ \mathcal{X}=\frac{i}{2} \big( H_{n-1} P_{n}\big) - \frac{i}{2} \big(P_{n}  H_{n-1}\big)$$

NOW multiply by $P_{n}$:

$$  P_{n} \mathcal{X}=\frac{i}{2} \big(P_{n} H_{n-1} P_{n}\big) - \frac{i}{2} \big(P_{n} P_{n}  H_{n-1}\big)$$

- $P_{n} P_{n}=\mathcal{I}$

$$  P_{n} \mathcal{X}=\frac{i}{2} \big(P_{n} H_{n-1} P_{n}\big) - \frac{i}{2} H_{n-1}$$

- $\left\{H_{n-1}, P_{n}\right\}=0$
    - $ H_{n-1} P_{n}= - P_{n} H_{n-1}$
    
$$  P_{n} \mathcal{X}= - \frac{i}{2} \big(P_{n} P_{n} H_{n-1}\big) - \frac{i}{2} H_{n-1}$$

$$  P_{n} \mathcal{X}= - \frac{i}{2} H_{n-1} - \frac{i}{2} H_{n-1}$$

$$  P_{n} \mathcal{X}= -i H_{n-1}$$
///////////////////////////////////////////////////////////

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} -i \sin \big(\frac{\alpha}{2}\big) \big(-i H_{n-1}\big)$$

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)H_{n-1}$$

note:
- $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$

- $ H_{n-1} =  \frac{S'_{n-1}}{\gamma_{l}} =  \sum_{k=1}^{n-1} \beta_{k} P_{k}$

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  \frac{S'_{n-1}}{\gamma_{l}}$$




rembering that our anti-commuting set was:
$$ S =\sum_{k=0}^{n-1} \beta_{j} P_{j}$$

- and $ S_{n-1} =\sum_{k=1}^{n-1} \beta_{j} P_{j}$ is $S$ with $\beta_{n} P_{n}$ REMOVED

we have:

$$ S = S_{n-1} + \beta_{n} P_{n}$$

hence:

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  \frac{S'_{n-1}}{\gamma_{l}}$$

as: $ S_{n-1} = \frac{S'_{n-1}}{\gamma_{l}}$

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  S_{n-1}$$

This looks very similar to: $ S = S_{n-1} + \beta_{n} P_{n}$

only with constants changed!!!

Taking:

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  S_{n-1}$$

$$ S = \beta_{n} P_{n} + S_{n-1}$$

we want $\langle S \rangle$... but we are getting $\langle P_{n} R \rangle$

(they are the same thing BUT with constants changed!)

therefore have:

$$ \Omega \big( \beta_{n} P_{n} + S_{n-1} \big) =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  S_{n-1}$$

Only lookin at the $P_{n}$ term:

$$ \Omega \beta_{n} P_{n} =  \cos \big( \frac{\alpha}{2} \big) P_{n} $$

$$ \Omega \beta_{n} =  \cos \big( \frac{\alpha}{2} \big) $$

$$ \Omega  =  {\cos \big( \frac{\alpha}{2} \big) \over \beta_{n}}$$

Overall we have:

$$\langle S \rangle = \Omega \langle P_{n} R \rangle$$


for $\langle R H_{n} R^{\dagger} \rangle=  \langle P_{n} \rangle$ we are actually measuring:

$$\langle R H_{n} \rangle =   \langle P_{n} R \rangle$$

$$ P_{n} R =  \cos \big( \frac{\alpha}{2} \big) P_{n} + \sin \big(\frac{\alpha}{2}\big)  H_{n-1}$$

S = set
$$ S =\sum_{k=0}^{n-1} \beta_{j} P_{j}$$
where all terms anti-commute: $ \{P_{j}P_{i},P_{i}P_{j}\}=0$

choosing a PauliWord in set: $ P_{n}$ and REMOVE IT

$$ S_{n-1} =\sum_{k=1}^{n-1} \beta_{k} P_{k}$$

note sum index starting from 1 now!

Next need amplitudes to sum to one... therefore have:

$$ S'_{n-1} = \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{k}$$

where:
- $ \beta_{j} = \gamma_{l} \times \beta_{k}$ 
- $ \sum_{k=1}^{n-1} \beta_{k}^{2} = 1$

NOW we define $H_{n-1}$:

$$ H_{n-1} =  \frac{S'_{n-1}}{\gamma_{l}} =  \sum_{k=1}^{n-1} \beta_{k} P_{k}$$


$$R=\cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X})$$

$$R H_{n} =  P_{n} R$$

$$R H_{n} = \bigg( \cos \big( \frac{\alpha}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha}{2}\big) \mathcal{X}\bigg) H_{n}$$

$$R H_{n} = \cos \big( \frac{\alpha}{2} \big) H_{n} -i \sin \big(\frac{\alpha}{2}\big) \mathcal{X}H_{n}$$

as:
- $\mathcal{X} H_{n}=i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$
- $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$
- AND **IMPORTANTLY**: $\alpha = \phi_{n-1}$ 

  $$R H_{n} = \cos \big( \frac{\alpha}{2} \big) \bigg(\sin (\alpha) H_{n-1}+\cos (\alpha) P_{n} \bigg) -i \sin \big(\frac{\alpha}{2}\big) \bigg(i (-\sin (\alpha) P_{n}+\cos (\alpha) H_{n-1}) \bigg)$$


$$R H_{n} R^{\dagger}=  P_{n}$$

$$R H_{n} =  P_{n} R$$

In [ ]:
# $$ \cos \big( \frac{\alpha}{2} \big)\sin (\alpha)+sin \big(\frac{\alpha}{2}\big)  \cos (\alpha) = \sin \big( \frac{3\alpha}{2} \big)$$


# $$ \cos \big( \frac{\alpha}{2} \big)\cos (\alpha)-\sin \big(\frac{\alpha}{2}\big)  \sin (\alpha) = \cos \big( \frac{\alpha}{2} \big)$$

$R H_{n} R^{\dagger}= \gamma_{l} \: \sin \left(\phi_{n-1}-\alpha \gamma_{l}\right) H_{n-1}+\cos \left(\phi_{n-1}-\alpha \gamma_{l}\right) P_{n}$

THEN choosing: $\alpha \gamma_{l} = \phi_{n-1}$ gives: $R H_{n} R^{\dagger}=P_{n}$

To find alpha it is simply:
$$\alpha \gamma_{l} = cos^{-1}(\beta_{s})$$

$$\alpha  = \frac{cos^{-1}(\beta_{s})}{\gamma_{l}} $$


In [9]:
chi= Get_X_SET(test_set, S_index)
chi

{'gamma_l': (0.04919764587885283+0j),
 'terms': [('X0 X1 X2 Y3', (1+0j))],
 'P_s': ('Z0 I1 I2 I3', (0.1371657293179602+0j))}



$$R=\exp (-i \frac{\alpha}{2} \mathcal{X})=\exp \bigg(-i \frac{\alpha}{2} \big(i \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{kn}\big) \bigg)$$

$$R=\exp \bigg(-i \frac{\alpha}{2} \big(\gamma_{l} \sum_{k=1}^{n-1} i \beta_{k} P_{kn}\big) \bigg) =\exp \bigg(-i \frac{\alpha \gamma_{l}}{2} \sum_{k=1}^{n-1} i \beta_{k} P_{kn} \bigg)$$

let: $V=\sum_{k=1}^{n-1} i \beta_{k} P_{kn}$

$$R=\exp \bigg(-i \frac{\alpha \gamma_{l}}{2} V \bigg)$$

$$R=\cos \big( \frac{\alpha \gamma_{l}}{2} \big) \mathbf{1}-i \sin \big(\frac{\alpha \gamma_{l}}{2}\big) V)$$

This operator has the following action:

with: $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$

as : $ H_{n-1} = \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{k}$

$$H_{n}=\sin(\phi_{n-1}) \: \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{k} + \cos \phi_{n-1} P_{n}$$

moving $\gamma_{l}$ for clarity:

$$H_{n}=\gamma_{l} \: \sin(\phi_{n-1})   \sum_{k=1}^{n-1} \beta_{k} P_{k} + \cos \phi_{n-1} P_{n}$$




$R H_{n} R^{\dagger}= \gamma_{l} \: \sin \left(\phi_{n-1}-\alpha \gamma_{l}\right) H_{n-1}+\cos \left(\phi_{n-1}-\alpha \gamma_{l}\right) P_{n}$

THEN choosing: $\alpha \gamma_{l} = \phi_{n-1}$ gives: $R H_{n} R^{\dagger}=P_{n}$

To find alpha it is simply:
$$\alpha \gamma_{l} = cos^{-1}(\beta_{s})$$

$$\alpha  = \frac{cos^{-1}(\beta_{s})}{\gamma_{l}} $$



## Paper verison

$\begin{aligned} \mathcal{X} H_{n} &=i H_{n-1} P_{n} H_{n} \\ &=i H_{n-1} P_{n}\left(\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}\right) \\ &=i\left(\sin \phi_{n-1} H_{n-1} P_{n} H_{n-1}+\cos \phi_{n-1} H_{n-1} P_{n}^{2}\right) \end{aligned}$

as $\left\{H_{n-1}, P_{n}\right\}=0$ and $P_{n}^{2}=1$

$$\mathcal{X} H_{n}=i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

therefore have:
$\left[\mathcal{X}, H_{n}\right]=2 i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$

$$\begin{aligned} e^{-i(\alpha / 2) A} B e^{i(\alpha / 2) A} &=\cos ^{2}(\alpha / 2) B+\sin ^{2}(\alpha / 2) A B A \\ &+i \sin (\alpha / 2) \cos (\alpha / 2)[A, B] \end{aligned}$$

for
- $A^{2}=\mathbb{1}$
- $e^{-i(\alpha / 2) A}= \cos (\alpha / 2) \mathbb{1}-i \sin (\alpha / 2) A)$

as: $R=e^{-i(\alpha / 2) \mathcal{X}}$

$$R H_{n} R^{\dagger}=\cos ^{2}(\alpha / 2) H_{n}+\sin ^{2}(\alpha / 2) \mathcal{X} H_{n} \mathcal{X} +i \sin (\alpha / 2) \cos (\alpha / 2)\left[\mathcal{X}, H_{n}\right]$$


$$=\left(\cos ^{2} \alpha / 2-\sin ^{2} \alpha / 2\right) H_{n} +(i / 2) 2 \sin (\alpha / 2) \cos (\alpha / 2)\left[\mathcal{X}, H_{n}\right]$$

$$= \cos \alpha H_{n}-\sin \alpha\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

$$=\cos \alpha\left(\cos \phi_{n-1} P_{n}+\sin \phi_{n-1} H_{n-1}\right)-\sin \alpha\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

$$=\left(\cos \alpha \cos \phi_{n-1}+\sin \alpha \sin \phi_{n-1}\right) P_{n} +\left(\cos \alpha \sin \phi_{n-1}-\sin \alpha \cos \phi_{n-1}\right) H_{n-1}$$

$$=\cos \left(\phi_{n-1}-\alpha\right) P_{n} +\sin \left(\phi_{n-1}-\alpha\right) H_{n-1}$$

## using MY version

$$\chi =i H_{n-1} P_{n} =i \big( \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{k} \big) P_{n} = i \gamma_{l} \sum_{k=1}^{n-1} \beta_{k} P_{kn}$$

$\begin{aligned} \mathcal{X} H_{n} &=i H_{n-1} P_{n} H_{n} \\ &=i H_{n-1} P_{n}\left(\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}\right) \\ &=i\left(\sin \phi_{n-1} H_{n-1} P_{n} H_{n-1}+\cos \phi_{n-1} H_{n-1} P_{n}^{2}\right) \end{aligned}$

as $\left\{H_{n-1}, P_{n}\right\}=0$ and $P_{n}^{2}=1$

$$\mathcal{X} H_{n}=i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

therefore have:
$\left[\mathcal{X}, H_{n}\right]=2 i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$

$\mathcal{X} H_{n} =i H_{n-1} P_{n} H_{n}$

$=i H_{n-1} P_{n}\left(\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}\right)$

$=i\left(\sin \phi_{n-1} H_{n-1} P_{n} H_{n-1}+\cos \phi_{n-1} H_{n-1} P_{n}^{2}\right)$

as $\left\{H_{n-1}, P_{n}\right\}=0$ and $P_{n}^{2}=1$

$$\mathcal{X} H_{n}=i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

therefore have:
$\left[\mathcal{X}, H_{n}\right]=2 i\left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$

$$\begin{aligned} e^{-i(\alpha / 2) A} B e^{i(\alpha / 2) A} &=\cos ^{2}(\alpha / 2) B+\sin ^{2}(\alpha / 2) A B A \\ &+i \sin (\alpha / 2) \cos (\alpha / 2)[A, B] \end{aligned}$$

for
- $A^{2}=\mathbb{1}$
- $e^{-i(\alpha / 2) A}= \cos (\alpha / 2) \mathbb{1}-i \sin (\alpha / 2) A)$

let: $V=\sum_{k=1}^{n-1} i \beta_{k} P_{kn}$ ... (note in paper this is how $\mathcal{X}$ is defined)

where:
- $V^{2}=\mathbb{1}$
- $e^{-i\frac{\alpha \gamma_{l}}{2} V}= \cos (\frac{\alpha \gamma_{l}}{2}) \mathbb{1}-i \sin (\frac{\alpha \gamma_{l}}{2}) V$

$R= e^{\big(-i\frac{\alpha \gamma_{l}}{2} V \big)}$  NEW!!!



$$R H_{n} R^{\dagger}=\cos ^{2}(\frac{\alpha \gamma_{l}}{2}) H_{n}+\sin ^{2}(\frac{\alpha \gamma_{l}}{2}) \mathcal{X} H_{n} \mathcal{X} +i \sin (\frac{\alpha \gamma_{l}}{2}) \cos (\frac{\alpha \gamma_{l}}{2})\left[\mathcal{X}, H_{n}\right]$$


$$=\left(\cos ^{2} \frac{\alpha \gamma_{l}}{2}-\sin ^{2} \frac{\alpha \gamma_{l}}{2} \right) H_{n} +(i / 2) 2 \sin (\frac{\alpha \gamma_{l}}{2}) \cos (\frac{\alpha \gamma_{l}}{2})\left[\mathcal{X}, H_{n}\right]$$

$$= \cos \alpha \gamma H_{n}-\sin \alpha \gamma \left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

$$=\cos \alpha \gamma \left(\cos \phi_{n-1} P_{n}+\sin \phi_{n-1} H_{n-1}\right)-\sin \alpha \gamma \left(-\sin \phi_{n-1} P_{n}+\cos \phi_{n-1} H_{n-1}\right)$$

$$=\left(\cos \alpha \gamma \cos \phi_{n-1}+\sin \alpha \gamma \sin \phi_{n-1}\right) P_{n} +\left(\cos \alpha \gamma \sin \phi_{n-1}-\sin \alpha \gamma \cos \phi_{n-1}\right) H_{n-1}$$

$$=\cos \left(\phi_{n-1}-\alpha \gamma \right) P_{n} +\sin \left(\phi_{n-1}-\alpha \gamma \right) H_{n-1}$$1

$$R=\exp (-i \alpha \mathcal{X} / 2)=\cos (\alpha / 2) \mathbf{1}-i \sin (\alpha / 2) \mathcal{X}$$

note it has the following action:

$R H_{n} R^{\dagger}=\sin \left(\phi_{n-1}-\alpha\right) H_{n-1}+\cos \left(\phi_{n-1}-\alpha\right) P_{n}$

with: $H_{n}=\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}$

THEN choosing: $\alpha = \phi_{n-1}$ gives: $R H_{n} R^{\dagger}=P_{n}$

To find alpha it is simply:
$$\alpha = cos^{-1}(\beta_{s})$$

$H_{n-1}=\sum_{k=1}^{n-1} \beta_{k} P_{k}$

In [5]:
Get_R_times_Hn_terms(LCU)

{'Z0 I1 I2 I3': (0.9852118646969333+0j), 'Y0 X1 X2 Y3': (-0.17134054295580894+0j)} VS {'Z0 I1 I2 I3': (0.9852118646969333+0j), 'Y0 X1 X2 Y3': (-0.17134054295580903-0j)}


[('Z0 I1 I2 I3', (0.9852118646969333+0j)),
 ('Y0 X1 X2 Y3', (-0.17134054295580894+0j))]

In [ ]:
# # ### Join Circuits
# # S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 1, 9: 1, 10: 1}
# # w=ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, 4, 1)
# # ####
# #
# # # simulate
# # tt = ALCU_Simulation_Quantum_Circuit_Dict(w, 100, 1)
# # tt.Get_expectation_value_via_parity()
# # tt.Calc_energy_via_parity()
# # tt.Energy
# #pys
# #
# #
# #
# #
# # S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 1, 9: 1, 10: 1}
# # def Calc_E_UP(THETA_params):
# #     ansatz_Q_cicuit = HF_UCCSD_ansatz.Get_Full_HF_UCCSD_QC(THETA_params)
# #     w = ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, 4, 1)
# #     tt = ALCU_Simulation_Quantum_Circuit_Dict_SHOTS(w, 1000, 1)
# #     tt.Get_expectation_value_via_parity()
# #     tt.Calc_energy_via_parity()
# #     return tt.Energy.real
# # from quchem.Scipy_Optimizer import *
# # THETA_params = [1, 2, 3]
# # GG = Optimizer(Calc_E_UP, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
# #                tol=1e-5,
# #                display_convergence_message=True)
# # GG.get_env(50)
# # GG.plot_convergence()
# # plt.show()


# ### Join Circuits

# # S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
# # w =ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped,QubitHam_PauliStr, S_dict, Hamilt.molecule.n_qubits,
# #                       number_ancilla_qubits)
# # ####
# #
# # # simulate
# # tt = ALCU_Simulation_Quantum_Circuit_DictRAW(w, 100, 1)
# # tt.Get_expectation_value_via_parity()
# # tt.Calc_energy_via_parity()
# # tt.Energy
# #
# #
# #
# #
# #
# S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
# def Calc_E_UP(THETA_params):
#     ansatz_Q_cicuit = HF_UCCSD_ansatz.Get_Full_HF_UCCSD_QC(THETA_params)
#     w = ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, Hamilt.molecule.n_qubits)
#     tt = ALCU_Simulation_Quantum_Circuit_DictRAW(w, 50_000)
#     tt.Get_expectation_value_via_parity()
#     tt.Calc_energy_via_parity()
#     return tt.Energy.real
# from quchem.Scipy_Optimizer import *

# import random
# THETA_params = [random.uniform(0,2*np.pi) for _ in range(len(THETA_params))]


# # THETA_params = [1, 2, 3]
# # THETA_params = [0.23333333, 3.13333333, 3.05]
# GG = Optimizer(Calc_E_UP, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
#                tol=1e-5,
#                display_convergence_message=True)
# GG.get_env(50)
# GG.plot_convergence()
# plt.show()






# ansatz_Q_cicuit = NEW_Anastz(np.pi)

# S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
# w =ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, Hamilt.molecule.n_qubits)
# ###
# # simulate
# tt = ALCU_Simulation_Quantum_Circuit_DictRAW(w, 50_000)
# tt.Get_expectation_value_via_parity()
# tt.Calc_energy_via_parity()
# tt.Energy

# S_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}



# def Calc_E_UP(THETA):
#     ansatz_Q_cicuit = NEW_Anastz(THETA)
#     w = ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, Hamilt.molecule.n_qubits)
#     tt = ALCU_Simulation_Quantum_Circuit_DictRAW(w, 500)
#     tt.Get_expectation_value_via_parity()
#     tt.Calc_energy_via_parity()
#     return tt.Energy.real
# from quchem.Scipy_Optimizer import *

# import random
# THETA = random.uniform(0,2*np.pi)
# #
# #
# # # THETA_params = [1, 2, 3]
# # # THETA_params = [0.23333333, 3.13333333, 3.05]
# # GG = Optimizer(Calc_E_UP, [THETA], 'Nelder-Mead', store_values=True, display_iter_steps=True,
# #                tol=1e-5,
# #                display_convergence_message=True)
# # GG.get_env(50)
# # GG.plot_convergence()
# # plt.show()


# E_list=[]
# from tqdm import tqdm
# for i in tqdm(np.arange(0, 2*np.pi, 0.05), ascii=True, desc='Getting ENERGIES'):

#     ansatz_Q_cicuit = NEW_Anastz(i)
#     w = ALCU_dict(ansatz_Q_cicuit, anti_commuting_set_stripped, S_dict, Hamilt.molecule.n_qubits)
#     ###
#     # simulate
#     tt = ALCU_Simulation_Quantum_Circuit_DictRAW(w, 100000)
#     tt.Get_expectation_value_via_parity()
#     tt.Calc_energy_via_parity()
#     E_list.append(tt.Energy)
# plt.plot(E_list)
# plt.show()
# print('VQE min E = {} VS FCI E = {}'.format(min(E_list), Hamilt.molecule.fci_energy))

In [ ]:
# $\mathcal{X} H_{n} =i H_{n-1} P_{n} H_{n}$

# $=i H_{n-1} P_{n}\left(\sin \phi_{n-1} H_{n-1}+\cos \phi_{n-1} P_{n}\right)$

# $=i\left(\sin \phi_{n-1} H_{n-1} P_{n} H_{n-1}+\cos \phi_{n-1} H_{n-1} P_{n}^{2}\right)$

# as $\left\{H_{n-1}, P_{n}\right\}=0$ and $P_{n}^{2}=1$